In [2]:
import os
from langchain_google_vertexai import ChatVertexAI
from langchain_core.pydantic_v1 import BaseModel, Field
from google.oauth2 import service_account

key_path = "jupyter_notebook_ai_clone/config/zen-general-377713-13db13cf4b80.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path

PROJECT_ID = 'zen-general-377713'
LOCATION = 'us-central1'
MODEL_NAME = "gemini-2.0-flash-001"

chat_model = ChatVertexAI(
    project_id=PROJECT_ID,
    location=LOCATION,
    model_name=MODEL_NAME,
    credentials=credentials
)

response = chat_model.invoke("hi")
print(response)
#content='Hi there! 👋  How can I help you today? 😊 \n' additional_kwargs={} response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.02978515625, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.0302734375}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.049560546875, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.048095703125}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.04345703125, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.0169677734375}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'probability_score': 0.140625, 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE', 'severity_score': 0.023681640625}], 'usage_metadata': {'prompt_token_count': 1, 'candidates_token_count': 15, 'total_token_count': 16, 'prompt_tokens_details': [{'modality': 1, 'token_count': 1}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 15}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.13779719670613608, 'model_name': 'gemini-1.5-flash-001'} id='run-8fe37d83-fb90-47bc-9790-b54dfe53fed8-0' usage_metadata={'input_tokens': 1, 'output_tokens': 15, 'total_tokens': 16}


/opt/conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3526: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


content='Hi there! How can I help you today?\n' additional_kwargs={} response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 1, 'candidates_token_count': 11, 'total_token_count': 12, 'prompt_tokens_details': [{'modality': 1, 'token_count': 1}], 'candidates_tokens_details': [{'modality': 1, 'token_count': 11}], 'cached_content_token_count': 0, 'cache_tokens_details': []}, 'finish_reason': 'STOP', 'avg_logprobs': -0.003992335362867875, 'model_name': 'gemini-2.0-flash-001'} id='run-12a50369-da28-40ba-a91d-e2fb810c104b-0' usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12}


In [5]:
import os
import base64
from langchain_google_vertexai import ChatVertexAI
from langchain_core.pydantic_v1 import BaseModel, Field
from google.oauth2 import service_account
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage

# Your existing Google Vertex AI setup
key_path = "jupyter_notebook_ai_clone/config/zen-general-377713-13db13cf4b80.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path

PROJECT_ID = 'zen-general-377713'
LOCATION = 'us-central1'
MODEL_NAME = "gemini-2.0-flash-001"

chat_model = ChatVertexAI(
    project_id=PROJECT_ID,
    location=LOCATION,
    model_name=MODEL_NAME,
    credentials=credentials
)

# Function to convert PDF to base64
def pdf_to_base64(pdf_path):
    with open(pdf_path, "rb") as pdf_file:
        encoded_string = base64.b64encode(pdf_file.read())
        return encoded_string.decode('utf-8')

# Load and convert PDF document
pdf_path = "jupyter_notebook_ai_clone/langgraph/data/sample_invoice.pdf"
base64_pdf = pdf_to_base64(pdf_path)

# Create a message with the base64-encoded PDF
def process_pdf_with_query(query, base64_pdf):
    message = HumanMessage(
        content=[
            {"type": "text", "text": query},
            {
                "type": "image_url",
                "image_url": {"url": f"data:application/pdf;base64,{base64_pdf}"},
            },
        ],
    )
    response = chat_model.invoke([message])
    return response.content

# Example query
query = "Extract the invoice number, date, total amount, and vendor name from this invoice"
result = process_pdf_with_query(query, base64_pdf)
print(result)

# If you want to process multiple PDFs, you can create a function for that
def process_multiple_pdfs(pdf_directory, query):
    results = {}
    for filename in os.listdir(pdf_directory):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_directory, filename)
            base64_pdf = pdf_to_base64(pdf_path)
            result = process_pdf_with_query(query, base64_pdf)
            results[filename] = result
    return results

# Example usage for multiple PDFs
# pdf_directory = "path/to/your/pdfs"
# query = "Extract the invoice number, date, and total amount"
# all_results = process_multiple_pdfs(pdf_directory, query)
# for filename, result in all_results.items():
#     print(f"--- Results for {filename} ---")
#     print(result)
#     print("\n")

Here's the information extracted from the invoice:

*   **Invoice Number:** INT-001
*   **Date:** 11/02/2019
*   **Total Amount:** \$204.75
*   **Vendor Name:** John Smith
